# Compute eddy kinetic energy from surface velocity anomalies
The anomaly datasets of merged surface velocities are now ready to be used to calculate EKE. This is done using cdo due to the size of the datasets. cdo will not change the metadata of the datasets, so the EKE data will still have the metadata of the original velocity datasets until it is changed. cdo also produces a new dataset in between each stage of the calculations, so to save space, it may be wise to delete the interim files.

In [ ]:
import xarray as xr
import numpy as np
from cdo import *
cdo=Cdo()
from joblib import Parallel, delayed
import gc

In [ ]:
#square the velocity anomalies
for i in [1,2,3,4,5]:
    inp='c6_ssh-vo_mon_lin_eq_5day_node_r'+str(i)+'_1860-2090_nol_anom.nc'
    outp='c6_ssh-vo2_mon_lin_eq_5day_node_r'+str(i)+'_1860-2090_nol_anom.nc'
    cdo.mul(input=([inp,inp]),output=outp)

In [ ]:
for i in [1,2,3,4,5]:
    inp='c6_ssh-uo_mon_lin_eq_5day_node_r'+str(i)+'_1860-2090_nol_anom.nc'
    outp='c6_ssh-uo2_mon_lin_eq_5day_node_r'+str(i)+'_1860-2090_nol_anom.nc'
    cdo.mul(input=([inp,inp]),output=outp)

In [ ]:
#add the zonal and merdional square velocities
for i in [1,2,3,4,5]:
    inv='c6_ssh-vo2_mon_lin_eq_5day_node_r'+str(i)+'_1860-2090_nol_anom.nc'
    inu='c6_ssh-uo2_mon_lin_eq_5day_node_r'+str(i)+'_1860-2090_nol_anom.nc'
    outp='c6_ssh-vo2+uo2_mon_lin_eq_5day_node_r'+str(i)+'_1860-2090_nol_anom.nc'
    cdo.add(input=([inv,inu]),output=outp)

In [ ]:
#multiply by 1/2
for i in [1,2,3,4,5]:
    inp='c6_ssh-vo2+uo2_mon_lin_eq_5day_node_r'+str(i)+'_1860-2090_nol_anom.nc'
    outp='c6_ssh-eke_mon_lin_eq_5day_node_r'+str(i)+'_1860-2090_nol_anom.nc'
    cdo.mulc(0.5,input=inp,output=outp)

In [ ]:
#calculate a one year running mean to mask seasonality in the time series
for i in [1,2,3,4,5]:
    inp='c6_ssh-eke_mon_lin_eq_5day_node_r'+str(i)+'_1860-2090_nol_anom.nc'
    outp='c6_ssh-eke_mon_lin_eq_5day_node_r'+str(i)+'_1860-2090_nol_anom_runmean.nc'
    cdo.runmean(73,input=inp,output=outp)

In [ ]:
#change the metadata of the resulting files
def namex_rm(i):
    temp=xr.open_dataset('c6_ssh-eke_mon_lin_eq_5day_node_r'+str(i)+'_1860-2090_nol_anom_runmean.nc')
    temp=temp.rename({'vo':'EKE'})
    temp['EKE']=temp.EKE.assign_attrs({'units':'m^2/s^2',
                                       'description':'Eddy kinetic energy computed using 5day mean geostrophic surface velocity data. Anomalies based on 21yr running mean for each 5 day period. 1 year running mean applied',
                                       'data':'historic-ssp370 simukations from AWICM1s CMIP6 contribution. r'+str(i)+' of five ensemble members'})
    
    temp.to_netcdf('c6_ssh-eke_mon_lin_eq_5day_node_r'+str(i)+'_1860-2090_nol_21yr_anom_runmean_rename.nc')

In [ ]:
Parallel(n_jobs=5,verbose=100)(delayed(namex_rm)(i) for i in [1,2,3,4,5])

In [ ]:
def namex(i):
    temp=xr.open_dataset('c6_ssh-eke_mon_lin_eq_5day_node_r'+str(i)+'_1860-2090_nol_anom.nc')
    temp=temp.rename({'vo':'EKE'})
    temp['EKE']=temp.EKE.assign_attrs({'units':'m^2/s^2',
                                       'description':'Eddy kinetic energy computed using 5day mean geostrophic surface velocity data. Anomalies based on 21yr running mean for each 5 day period.',
                                       'data':'historic-ssp370 simukations from AWICM1s CMIP6 contribution. r'+str(i)+' of five ensemble members'})
    
    temp.to_netcdf('c6_ssh-eke_mon_lin_eq_5day_node_r'+str(i)+'_1860-2090_nol_21yr_anom_rename.nc')
    
    temp.close()
    gc.collect(2)
    gc.collect(1)

In [ ]:
Parallel(n_jobs=3,verbose=100)(delayed(namex)(i) for i in [1,2,3,4,5])

In [ ]:
#calculate the ensemble mean (only necessary for the non-running mean data)
files=[]
outp='c6_ssh-eke_mon_lin_eq_5day_node_ensmean_1860-2090_nol_21yr_anom_rename.nc'
for i in [1,2,3,4,5]:
    files.append('c6_ssh-eke_mon_lin_eq_5day_node_r'+str(i)+'_1860-2090_nol_21yr_anom_rename.nc')
cdo.ensmean(input=(files),output=outp)